In [66]:
# Import libraries
import os
import imgaug.augmenters as iaa
import cv2 as cv
from moviepy.editor import VideoFileClip


## Train data pre-processing
#### Convert all images to .jpg and change all file name to sequential file name
Trained images were accquired from the internet by scraped technique. the name of imeages are random and not in order, and are not in same format.

For ease of processing, They will all be converted and changed name by creating function below.

In [67]:
# create function to convert all images format to .jpg
def convert_to_jpg(folder_path):
    # Check if the folder exists
    if not os.path.exists(folder_path):
        print(f"Folder '{folder_path}' not found.")
        return

    # List all files in the folder
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Sort files alphabetically
    files.sort()

    # Rename files to sequential names
    for index, file_name in enumerate(files):
        # Get the file extension
        file_extension = os.path.splitext(file_name)[1]

        # Construct the new file name
        new_file_name = f"{index + 1}.jpg"

        # Construct the new file path
        old_file_path = os.path.join(folder_path, file_name)
        new_file_path = os.path.join(folder_path, new_file_name)

        try:
            # Rename the file
            os.rename(old_file_path, new_file_path)
            print(f"File successfully renamed: {file_name} -> {new_file_name}")
        except Exception as e:
            print(f"Error renaming file {file_name}: {e}")

In [132]:
# execute function
path_ambulance = '../data/train_image/ambulance/'
path_curtain = '../data/train_image/curtain/'
convert_to_jpg(path_curtain)

File successfully renamed: 1.jpg -> 1.jpg
Error renaming file 10.jpg: [WinError 183] Cannot create a file when that file already exists: '../data/train_image/curtain/10.jpg' -> '../data/train_image/curtain/2.jpg'
Error renaming file 100.jpg: [WinError 183] Cannot create a file when that file already exists: '../data/train_image/curtain/100.jpg' -> '../data/train_image/curtain/3.jpg'
Error renaming file 101.jpg: [WinError 183] Cannot create a file when that file already exists: '../data/train_image/curtain/101.jpg' -> '../data/train_image/curtain/4.jpg'
Error renaming file 102.jpg: [WinError 183] Cannot create a file when that file already exists: '../data/train_image/curtain/102.jpg' -> '../data/train_image/curtain/5.jpg'
Error renaming file 103.jpg: [WinError 183] Cannot create a file when that file already exists: '../data/train_image/curtain/103.jpg' -> '../data/train_image/curtain/6.jpg'
Error renaming file 104.jpg: [WinError 183] Cannot create a file when that file already exists:

In [133]:
# Function to check if image format is compatible with tensorflow

from pathlib import Path
import imghdr

data_dir = path_curtain  # change path folder here
image_extensions = [".png", ".jpg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")

#### Image augmentation
The selected image may appear in video in different angle, therefore the trained image should be in multiple angle and size

In [78]:
# Create a list of augmentation techniques
augmentations = [
    iaa.Fliplr(p=1.0), # horizontal flips 
    iaa.Flipud(p=1.0), # vertical flips 
    iaa.Crop(percent=(0, 0.1)), # random crops 
    iaa.Affine(rotate=(0, 0)), # rotate 
    iaa.Affine(rotate=(3, 42)), # rotate 
    iaa.Affine(rotate=(48, 87)), # rotate 
    iaa.Affine(rotate=(93, 132)), # rotate 
    iaa.Affine(rotate=(138, 177)), # rotate 
    iaa.Affine(rotate=(222, 267)), # rotate 
    iaa.Affine(rotate=(273, 357)), # rotate     
    iaa.GaussianBlur(sigma=(0, 0.5)), # gaussian blur 
]


In [79]:
# create function to augment images and save them as new file
def augment_images(file1, file2,augment, k):
    # Read the image
    image = cv.imread(file1)

    # Augment the image
    image_augmented = augment(image=image)
    
    # Save the image
    cv.imwrite(f"{file2[:-4] + str(augment)[0:4] + str(k)}_augmented.jpg", image_augmented)

In [136]:
# Using loop to execute the augment function and save images
folder_path = '../data/train_image/curtain/'  #ambulance and curtain
folder_path_new = '../data/train_image/augmented/curtain/'  #ambulance and curtain
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

for file in files:
    file1 = folder_path + file
    file2 = folder_path_new + file
    for k in range(len(augmentations)):
        augment_images(file1, file2, augmentations[k], k)

Increase image by crop all of the image in different size

In [137]:
# Create a list of augmentation techniques
augmentations_2 = [ 
    iaa.Crop(percent=(0, 0.1)), # random crops
    iaa.Crop(percent=(0, 0.25)), # random crops
    iaa.Crop(percent=(0, 0.50)), # random crops
    iaa.Crop(percent=(0, 0.75)), # random crops
]

# repeat the same process for the second set of augmentations
folder_path = '../data/train_image/augmented/curtain/'  #ambulance and curtain
folder_path_new = '../data/train_image/augmented/curtain/'  #ambulance and curtain
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

for file in files:
    file1 = folder_path + file
    file2 = folder_path_new + file
    for k in range(len(augmentations_2)):
        augment_images(file1, file2, augmentations_2[k], k)

## Input conversion

The input is viedeo. This code will chop video into multiple picture. Using the first frame of each seconds of video and store them as picture for prediction

In [103]:
# Define vidoes path
file = '../data/test_video/2.mp4'
duration = VideoFileClip(file).duration
video = cv.VideoCapture(file)
fps = round(video.get(cv.CAP_PROP_FPS))
frame_id = int(duration * fps)



In [104]:
# Create function to extract frames from video and save them as new file name 1.jpg, 2.jpg, etc.
def v_to_frame(file, frame_id,k):
    video = cv.VideoCapture(file)
    video.set(cv.CAP_PROP_POS_FRAMES, frame_id)
    ret, frame = video.read()    
    cv.imwrite(f'../data/test_data/test_set2/{k}.jpg', frame)


In [105]:
# using loop to extract 1st frame of every second of video 
j=1
for i in range(0, frame_id+1, fps):
        v_to_frame(file, i, j)
        j+=1

